<h1 align=center>Capítulo 2</h1>
<h2 align=center>Strings e Texto</h2>
<p align=center><img src=https://www.otempo.com.br/image/contentid/policy:1.2743412:1664665976/uso-de-smartphone-e-celular050720211240-jpg.jpg?f=3x2&w=1224 width=500</p>

Quase todo programa útil envolve algum tipo de processamento de texto, seja analisando dados ou gerando saída. Este capítulo se concentra em problemas comuns envolvendo manipulação de texto, como separar strings, pesquisar, substituir, lexing e parsing. Muitas dessas tarefas podem ser facilmente resolvidas usando métodos integrados de strings. No entanto, operações mais complicadas podem exigir o uso de expressões regulares ou a criação de um analisador completo. Todos esses tópicos são abordados. Além disso, alguns aspectos complicados de trabalhar com *Unicode* são abordados.

## 2.1. Dividindo strings em qualquer um dos vários delimitadores

**Problema**

Você precisa dividir uma string em campos, mas os delimitadores (e o espaçamento entre eles) não são consistentes em toda a string.

**Solução**

O método `split()` de objetos string é realmente destinado a casos muito simples e não permite vários delimitadores ou conta para possíveis espaços em branco ao redor dos delimitadores. Nos casos em que você precisa de um pouco mais de flexibilidade, use o método `re.split()`:

In [185]:
line = 'asdf fjdk; afed, fjek,asdf, foo'

import re
re.split(r'[:,\s]\s*', line)

['asdf', 'fjdk;', 'afed', 'fjek', 'asdf', 'foo']

**Discussão**

A função `re.split()` é útil porque você pode especificar vários padrões para o separador. Por exemplo, conforme mostrado na solução, o separador é uma vírgula (,), ponto e vírgula (;) ou espaço em branco seguido por qualquer quantidade de espaço em branco extra. Sempre que esse padrão é encontrado, a correspondência inteira se torna o delimitador entre quaisquer campos que estejam em ambos os lados da correspondência. O resultado é uma lista de campos, assim como com `str.split()`.

Ao usar `re.split()`, você precisa ter um pouco de cuidado caso o padrão de expressão regular envolva um grupo de captura entre parênteses. Se forem usados grupos de captura, o texto correspondente também será incluído no resultado. Por exemplo, observe o que acontece aqui:

In [186]:
fields = re.split(r'(;|,|\s)\s*', line)
fields

['asdf', ' ', 'fjdk', ';', 'afed', ',', 'fjek', ',', 'asdf', ',', 'foo']

Obter os caracteres divididos pode ser útil em determinados contextos. Por exemplo, talvez você precise dos caracteres divididos posteriormente para reformar uma string de saída:

In [187]:
values = fields[::2]
delimiters = fields[1::2] + ['']
values

['asdf', 'fjdk', 'afed', 'fjek', 'asdf', 'foo']

In [188]:
delimiters

[' ', ';', ',', ',', ',', '']

In [189]:
# Reforma a linha usando os mesmos delimitadores
''.join(v+d for v,d in zip(values, delimiters))

'asdf fjdk;afed,fjek,asdf,foo'

Se você não quiser os caracteres separadores no resultado, mas ainda precisar usar parênteses para agrupar partes do padrão de expressão regular, certifique-se de usar um grupo sem captura, especificado como (?:...). Por exemplo:

In [190]:
re.split(r'(?:,|;|\s)\*',line)

['asdf fjdk; afed, fjek,asdf, foo']

## 2.2. Texto correspondente no início ou no final de uma string

**Problema**

Você precisa verificar o início ou o fim de uma string para padrões de texto específicos, como extensões de nome de arquivo, esquemas de URL e assim por diante.

**Solução**

Uma maneira simples de verificar o início ou o fim de uma string é usar os métodos `str.startswith()` ou `str.endswith()`. Por exemplo:

In [191]:
filename ='spam.txt'
filename.endswith('.txt')

True

In [192]:
filename.startswith('file:')

False

In [193]:
url = 'http://python.org'
url.startswith('http')

True

Se você precisar verificar várias opções, simplesmente forneça uma tupla de possibilidades para `startswith()` ou `endswith()`:

In [194]:
import os
filenames = os.listdir('.')
filenames

['.git',
 '.idea',
 '.ipynb_checkpoints',
 'Capítulo 1 - Estruturas de Dados e Algoritmos.ipynb',
 'Capítulo 2 - Strings e Texto.ipynb',
 'LICENSE',
 'README.md',
 'somefile.txt']

In [195]:
[name for name in filenames if name.endswith(('E', 't'))]

['.git', 'LICENSE', 'somefile.txt']

In [196]:
any(name.startswith('.ipynb') for name in filenames)

True

In [197]:
# Aqui outro exemplo

from urllib.request import urlopen

def read_data(name):
    if name.startswith(('http:', 'https:', 'ftp:')):
        return urlopen(name).read()
    else:
        with open(name) as f:
            return f.read()


Estranhamente, essa é uma parte do Python em que uma tupla é realmente necessária como entrada. Se você tiver as opções especificadas em uma lista ou conjunto, apenas certifique-se de convertê-las usando `tuple()` primeiro. Por exemplo:

In [198]:
choices = ['http:', 'ftp:']
url = 'http://www.python.org'

In [199]:
# url.startswith(choices)

#TypeError: startswith first arg must be str or a tuple of str, not list

In [200]:
url.startswith(tuple(choices))

True

**Discussão**

Os métodos `startswith()` e `endswith()` fornecem uma maneira muito conveniente de realizar a verificação básica de prefixos e sufixos. Operações semelhantes podem ser realizadas com fatias, mas são muito menos elegantes. Por exemplo:

In [201]:
filename = 'spam.txt'
filename[-4:] == '.txt'

True

In [202]:
url = 'http://www.python.org'
url[:5] == 'http:' or url[:6] == 'https:' or url[:4] == 'ftp:'

True

Você também pode estar inclinado a usar expressões regulares como alternativa. Por exemplo:

In [203]:
import re
url = 'http://www.python.org'
re.match('http:|https:|ftp:',url)

<re.Match object; span=(0, 5), match='http:'>

Isso funciona, mas geralmente é **um exagero para correspondência simples**. Usar esta receita é mais simples e funciona mais rápido.

Por último, mas não menos importante, os métodos `startswith()` e `endswith()` ficam bonitos quando combinados com outras operações, como reduções de dados comuns. Por exemplo, esta declaração que verifica um diretório quanto à presença de certos tipos de arquivos:

<code>
if any(name.endswith(('.c', '.h')) for name in listdir(dirname)):
    ...
</code>

## 2.3. Correspondência de strings usando padrões de caractere curinga do Shell

**Problema**

Você deseja corresponder o texto usando os mesmos padrões curinga que são comumente usados ao trabalhar em *shells Unix* (por exemplo, *.py, Dat\[0-9\]*.csv, etc.).

**Solução**

O módulo `fnmatch` fornece duas funções— `fnmatch()` e `fnmatchcase()` — que podem ser usadas para realizar tal correspondência. O uso é simples:

In [204]:
from fnmatch import fnmatch, fnmatchcase
fnmatch('foo.txt', '*.txt')

True

In [205]:
fnmatch('foo.txt', '?oo.txt')

True

In [206]:
fnmatch('Dat45.csv', 'Data[0-9]*')

False

In [207]:
names = ['Dat1.csv', 'Dat2.csv', 'config.ini', 'foo.py']
[name for name in names if fnmatch(name, 'Dat*.csv')]

['Dat1.csv', 'Dat2.csv']

Normalmente, `fnmatch()` corresponde a padrões usando as mesmas regras de distinção entre maiúsculas e minúsculas que o sistema de arquivos subjacente do sistema (que varia de acordo com o sistema operacional). Por exemplo:

In [208]:
# Em MacOs
fnmatch('foo.txt','*.TXT')

# Retornaria False

True

In [209]:
# Em Windows
fnmatch('foo.txt','*.TXT')

True

Se essa distinção for importante, use `fnmatchcase()` em vez disso. Ele corresponde exatamente com base nas convenções de letras minúsculas e maiúsculas que você fornece:

In [210]:
fnmatchcase('foo.txt', '*.TXT')

False

Um recurso muitas vezes esquecido dessas funções é seu uso potencial com processamento de dados de strings que não sejam nomes de arquivo. Por exemplo, suponha que você tenha uma lista de endereços como esta:

In [211]:
addresses = [
    '5412 N CLARK ST',
    '1060 W ADDISON ST',
    '1039 W GRANVILLE AVE',
    '2122 N CLARK ST',
    '4802 N BROADWAY',
]

In [212]:
# Você poderia escrever compreensões de lista como esta:
from fnmatch import fnmatchcase
[addr for addr in addresses if fnmatchcase(addr, '*ST')]

['5412 N CLARK ST', '1060 W ADDISON ST', '2122 N CLARK ST']

In [213]:
[addr for addr in addresses if fnmatchcase(addr, '54[0-9][0-9] * CLARK *')]

['5412 N CLARK ST']

**Discussão**

A correspondência realizada pelo `fnmatch` fica em algum lugar entre a funcionalidade dos métodos de string simples e o poder total das expressões regulares. Se você está apenas tentando fornecer um mecanismo simples para permitir curingas nas operações de processamento de dados, geralmente é uma solução razoável.

Se você está realmente tentando escrever um código que corresponda a nomes de arquivos, use o módulo `glob`. Veja a Receita 5.13.

## 2.4. Correspondência e pesquisa de padrões de texto

**Problema**

Você deseja corresponder ou pesquisar texto para um padrão específico.

**Solução**

Se o texto que você está tentando corresponder for um literal simples, muitas vezes você pode usar apenas os métodos básicos de string, como `str.find()`, `str.endswith()`, `str.startswith()` ou similares. Por exemplo:

In [214]:
text = 'yeah, but no, but yeah, but no, but yeah'

# Combinação exata
text == 'yeah'


False

In [215]:
# Combina no começo ou no final
text.startswith('yeah')

True

In [216]:
text.endswith('no')

False

In [217]:
# Procura a localização da primeira ocorrência
text.find('no')

10

Para correspondências mais complicadas, use expressões regulares e o módulo `re`. Para ilustrar a mecânica básica do uso de expressões regulares, suponha que você queira corresponder datas especificadas como dígitos, como “27/11/2012”. Aqui está uma amostra de como você faria isso:

In [218]:
text1 = '27/11/2012'
text2 = '27 Nov, 2012'

import re
# Combinação simple: \d+ significa um ou mais dígitos

if re.match(r'\d+/\d+/\d+', text1):
    print('Yes')

else:
    print('No')

Yes


In [219]:
if re.match(r'\d+/\d+/\d+', text2):
    print('Yes')

else:
    print('No')

No


Se você for realizar muitas correspondências usando o mesmo padrão, geralmente vale a pena pré-compilar o padrão de expressão regular em um objeto de padrão primeiro. Por exemplo:

In [220]:
datepat = re.compile(r'\d+/\d+/\d+')
if datepat.match(text1):
    print('yes')
else:
    print('no')

yes


In [221]:
if datepat.match(text2):
    print('yes')
else:
    print('no')

no


`match()` sempre tenta encontrar a correspondência no início de uma string. Se você quiser pesquisar texto para todas as ocorrências de um padrão, use o método `findall()`. Por exemplo:

In [222]:
text = 'Today is 11/27/2012. PyCon starts 3/13/2013.'
datepat.findall(text)

['11/27/2012', '3/13/2013']

Ao definir expressões regulares, é comum introduzir grupos de captura colocando partes do padrão entre parênteses. Por exemplo:

In [223]:
datepat = re.compile(r'(\d+)/(\d+)/(\d+)')

Os grupos de captura geralmente simplificam o processamento subsequente do texto correspondente porque o conteúdo de cada grupo pode ser extraído individualmente. Por exemplo:

In [224]:
m = datepat.match('11/27/2012')
m

<re.Match object; span=(0, 10), match='11/27/2012'>

In [225]:
# Extraindo o conteúdo de cada grupo
m.group(0)

'11/27/2012'

In [226]:
m.group(1)

'11'

In [227]:
m.group(2)

'27'

In [228]:
m.group(3)

'2012'

In [229]:
m.groups()

('11', '27', '2012')

In [230]:
mes, dia, ano = m.groups() # Empacotando

In [231]:
text

'Today is 11/27/2012. PyCon starts 3/13/2013.'

In [232]:
datepat.findall(text)

[('11', '27', '2012'), ('3', '13', '2013')]

In [233]:
for mes, dia, ano in datepat.findall(text):
    print(f"{ano}-{mes}-{dia}")

2012-11-27
2013-3-13


O método `findall()` pesquisa o texto e encontra todas as correspondências, retornando-as como uma lista. Se você quiser encontrar correspondências iterativamente, use o método `finditer()`. Por exemplo:

In [234]:
for m in datepat.finditer(text):
    print(m.groups())

('11', '27', '2012')
('3', '13', '2013')


**Discussão**

No entanto, esta receita ilustra o básico absoluto do uso do módulo `re` para combinar e pesquisar texto. A funcionalidade essencial é primeiro compilar um padrão usando `re.compile()` e depois usar métodos como `match()`, `findall()` ou `finditer()`.

Ao especificar padrões, é relativamente comum usar strings brutas como `r'(\d+)/(\d+)/(\d+)'`. Essas strings deixam o caractere de barra invertida sem interpretação, o que pode ser útil no contexto de expressões regulares. Caso contrário, você precisará usar barras invertidas duplas, como `'(\\d+)/(\\d+)/(\\d+)'`.

Esteja ciente de que o método `match()` verifica apenas o início de uma string. É possível que corresponda a coisas que você não está esperando. Por exemplo:

In [235]:
m = datepat.match('11/27/2012abcdef')
m

<re.Match object; span=(0, 10), match='11/27/2012'>

In [236]:
m.group()

'11/27/2012'

Se você quiser uma correspondência exata, certifique-se de que o padrão inclua o marcador final (`$`), como a seguir:

In [237]:
datepat = re.compile(r'(\d+)/(\d+)/(\d+)$')

In [238]:
datepat.match('11/27/2012abcdef')

In [239]:
datepat.match('11/27/2012')

<re.Match object; span=(0, 10), match='11/27/2012'>

Por último, se você está apenas fazendo uma operação simples de correspondência/pesquisa de texto, muitas vezes você pode pular a etapa de compilação e usar funções de nível de módulo no módulo `re`. Por exemplo:

In [240]:
re.findall(r'(\d+)/(\d+)/(\d+)', text)

[('11', '27', '2012'), ('3', '13', '2013')]

Esteja ciente, porém, de que, se você for realizar muitas correspondências ou pesquisas, geralmente vale a pena compilar o padrão primeiro e usá-lo repetidamente. As funções de nível de módulo mantêm um cache de padrões compilados recentemente, portanto, não há um grande impacto no desempenho, mas você economizará algumas pesquisas e processamento extra usando seu próprio padrão compilado.

## 2.5. Pesquisando e substituindo texto

**Problema**

Você deseja pesquisar e substituir um padrão de texto em uma string.

**Solução**

Para padrões literais simples, use o método `str.replace()`. Por exemplo:

In [241]:
text = 'yeah, but no, but yeah, but no, but yeah'
text.replace('yeah', 'sim')

'sim, but no, but sim, but no, but sim'

Para padrões mais complicados, use as funções/métodos `sub()` no módulo `re`. Para ilustrar, suponha que você queira reescrever datas no formato “27/11/2012” como “2012-11-27”. Aqui está uma amostra de como fazê-lo:

In [242]:
text = 'Today is 11/27/2012. PyCon starts 3/13/2013.'

import re
re.sub(r'(\d+)/(\d+)/(\d+)', r'\3-\1-\2', text)

'Today is 2012-11-27. PyCon starts 2013-3-13.'

O primeiro argumento para `sub()` é o padrão a ser correspondido e o segundo argumento é o padrão de substituição. Dígitos com barra invertida, como **\3**, referem-se aos números do grupo de captura no padrão.

Se você for realizar substituições repetidas do mesmo padrão, considere compilá-lo primeiro para obter um melhor desempenho. Por exemplo:

In [243]:
from calendar import month_abbr
def change_date(m):
    mon_name = month_abbr[int(m.group(1))]
    return f"{m.group(2)} {mon_name} {m.group(3)}"

datepat.sub(change_date, text)

'Today is 11/27/2012. PyCon starts 3/13/2013.'

Como entrada, o argumento para o retorno de chamada de substituição é um objeto de correspondência, conforme retornado por `match()` ou `find()`. Use o método `.group()` para extrair partes específicas da correspondência. A função deve retornar o texto de substituição. Se você quiser saber quantas substituições foram feitas além de obter o texto de substituição, use `re.subn()`. Por exemplo:

In [244]:
newtext, n = datepat.subn(r'\3-\1-\2', text)
newtext

'Today is 11/27/2012. PyCon starts 3/13/2013.'

In [245]:
n

0

**Discussão**

Não há muito mais para pesquisar e substituir expressões regulares do que os métodos `sub()` mostrados. A parte mais complicada é especificar o padrão de expressão regular – algo que é melhor deixar como exercício para o leitor.

## 2.6. Pesquisando e substituindo texto que não diferencia maiúsculas de minúsculas

**Problema**

Você precisa pesquisar e possivelmente substituir o texto sem diferenciar maiúsculas de minúsculas.

**Solução**

Para executar operações de texto que não diferenciam maiúsculas de minúsculas, você precisa usar o módulo `re` e fornecer o sinalizador `re.IGNORECASE` para várias operações. Por exemplo:

In [246]:
text = 'UPPER PYTHON, lower python, Mixed Python'
re.findall('python', text, flags=re.I) #re.I ou re.IGNORECASE

['PYTHON', 'python', 'Python']

In [247]:
re.sub('python', 'snake', text, flags = re.IGNORECASE)

'UPPER snake, lower snake, Mixed snake'

O último exemplo revela uma limitação de que a substituição de texto não corresponderá ao caso do texto correspondente. Se você precisar corrigir isso, talvez seja necessário usar uma função de suporte, como a seguir:

In [248]:
def matchcase(word):
    def replace(m):
        text = m.group()
        if text.isupper():
            return word.upper()
        elif text.islower():
            return word.lower()
        elif text[0].isupper():
            return word.capitalize()
        else:
            return word
    return replace

Aqui está um exemplo de uso desta última função:

In [249]:
re.sub('python', matchcase('snake'), text, flags=re.IGNORECASE)

'UPPER SNAKE, lower snake, Mixed Snake'

**Discussão**

Para casos simples, basta fornecer o `re.IGNORECASE` para realizar a correspondência que não diferencia maiúsculas de minúsculas. No entanto, esteja ciente de que isso pode não ser suficiente para certos tipos de correspondência Unicode envolvendo dobragem de maiúsculas e minúsculas. Veja a Receita 2.10 para mais detalhes.

## 2.7. Especificando uma expressão regular para a correspondência mais curta

**Problema**

Você está tentando corresponder a um padrão de texto usando expressões regulares, mas está identificando as correspondências mais longas possíveis de um padrão. Em vez disso, você gostaria de alterá-lo para encontrar a correspondência mais curta possível.

**Solução**

Esse problema geralmente surge em padrões que tentam corresponder o texto dentro de um par de delimitadores inicial e final (por exemplo, uma string entre aspas). Para ilustrar, considere este exemplo:

In [250]:
str_pat = re.compile(r'\"(.*)\"')
text1 = 'Computer says "no."'

str_pat.findall(text1)

['no.']

In [251]:
text2 = 'Computer says "no." Phone says "yes."'
str_pat.findall(text2)

['no." Phone says "yes.']

Neste exemplo, o padrão `r'\"(.*)\"'` está tentando corresponder ao texto entre aspas. No entanto, o operador * em uma expressão regular é ganancioso, portanto, a correspondência é baseada em encontrar a correspondência mais longa possível. 

Assim, no segundo exemplo envolvendo **text2**, ele corresponde incorretamente às duas strings entre aspas. Para corrigir isso, adicione o **?** modificador após o operador **\*** no padrão, assim:

In [252]:
str_pat = re.compile(r'\"(.*?)\"')
str_pat.findall(text2)

['no.', 'yes.']

Isso torna a correspondência não gananciosa e produz a correspondência mais curta.

**Discussão**

Esta receita aborda um dos problemas mais comuns encontrados ao escrever expressões regulares envolvendo o caractere ponto (.). Em um padrão, o ponto corresponde a qualquer caractere, exceto uma nova linha. No entanto, se você colocar entre parênteses o ponto com texto inicial e final (como uma citação), a correspondência tentará encontrar a correspondência mais longa possível para o padrão. Isso faz com que várias ocorrências do texto inicial ou final sejam ignoradas e incluídas nos resultados da correspondência mais longa. Adicionando o **?** logo após operadores como **\*** ou **+** força o algoritmo de correspondência a procurar a correspondência mais curta possível.

## 2.8. Escrevendo uma expressão regular para padrões de várias linhas

**Problema**

Você está tentando corresponder a um bloco de texto usando uma expressão regular, mas precisa que a correspondência abranja várias linhas.

**Solução**

Esse problema geralmente surge em padrões que usam o ponto **(.)** para corresponder a qualquer caractere, mas esquecem de levar em conta o fato de que ele não corresponde a novas linhas. Por exemplo, suponha que você esteja tentando corresponder comentários no estilo C:

In [253]:
comment = re.compile(r'/\*(.*?)\*/')
text1 = '/* this is a comment */'
text2 = '''/* this is a
            multiline comment */
            '''

comment.findall(text1)

[' this is a comment ']

In [254]:
comment.findall(text2)

[]

Para corrigir o problema, você pode adicionar suporte para novas linhas. Por exemplo:

In [255]:
comment = re.compile(r'/\*((?:.|\n)*?)\*/')
comment.findall(text2)

[' this is a\n            multiline comment ']

Nesse padrão, (?:.|\n) especifica um grupo de não captura (ou seja, define um grupo para fins de correspondência, mas esse grupo não é capturado separadamente ou numerado).

**Discussão**

A função `re.compile()` aceita um sinalizador, `re.DOTALL`, que é útil aqui. Faz o `.` em uma expressão regular corresponde a todos os caracteres, incluindo novas linhas. Por exemplo:

In [256]:
comment = re.compile(r'/\*(.*?)\*/', re.DOTALL)
comment.findall(text2)

[' this is a\n            multiline comment ']

Usar o sinalizador `re.DOTALL` funciona bem para casos simples, mas pode ser problemático se você estiver trabalhando com padrões extremamente complicados ou uma mistura de expressões regulares separadas que foram combinadas para fins de tokenização, conforme descrito na Receita 2.18. Se for dada uma escolha, geralmente é melhor definir seu padrão de expressão regular para que ele funcione corretamente sem a necessidade de sinalizadores extras.

## 2.9. Normalizando texto Unicode para uma representação padrão

**Problema**

Você está trabalhando com strings Unicode, mas precisa garantir que todas as strings tenham a mesma representação subjacente.

**Solução**

Em Unicode, certos caracteres podem ser representados por mais de uma sequência válida de pontos de código. Para ilustrar, considere o seguinte exemplo:

In [257]:
s1 = 'Spicy Jalape\u00f1o'
s2 = 'Spicy Jalapen\u0303o'

In [258]:
s1

'Spicy Jalapeño'

In [259]:
s2

'Spicy Jalapeño'

In [260]:
s1 == s2

False

In [261]:
len(s1)

14

In [262]:
len(s2)

15

Aqui o texto “Spicy Jalapeño” foi apresentado em duas formas. O primeiro usa o caractere “ñ” totalmente composto (U+00F1). O segundo usa a letra latina “n” seguida por um caractere de combinação “~” (U+0303).

Ter múltiplas representações é um problema para programas que comparam strings. Para corrigir isso, você deve primeiro normalizar o texto em uma representação padrão usando o módulo `unicodedata`:

In [263]:
import unicodedata

t1 = unicodedata.normalize('NFC', s1)
t2 = unicodedata.normalize('NFC', s2)

In [264]:
t1 == t2

True

In [265]:
print(ascii(t1))

'Spicy Jalape\xf1o'


In [266]:
t3 = unicodedata.normalize('NFD', s1)
t4 = unicodedata.normalize('NFD', s2)

t3 == t4

True

In [267]:
print(ascii(t3))

'Spicy Jalapen\u0303o'


O primeiro argumento para `normalize()` especifica como você deseja que a string seja normalizada. **NFC** significa que os caracteres devem ser totalmente compostos (ou seja, use um único ponto de código, se possível).
**NFD** significa que os caracteres devem ser totalmente decompostos com o uso de caracteres combinados.

Python também suporta os formulários de normalização **NFKC** e **NFKD**, que adicionam recursos extras de compatibilidade para lidar com certos tipos de caracteres. Por exemplo:

In [268]:
s = '\ufb01' # Caracter simples
s

'ﬁ'

In [269]:
unicodedata.normalize('NFD', s)

'ﬁ'

In [270]:
# Observe como as letras combinadas são separadas aqui
unicodedata.normalize('NFKD', s)

'fi'

In [271]:
unicodedata.normalize('NFKC', s)

'fi'

**Discussão**

A normalização é uma parte importante de qualquer código que precise garantir que processe texto Unicode de maneira sã e consistente. Isso é especialmente verdadeiro ao processar strings recebidas como parte da entrada do usuário, onde você tem pouco controle da codificação.

A normalização também pode ser uma parte importante da limpeza e filtragem de texto. Por exemplo, suponha que você queira remover todos os sinais diacríticos de algum texto (possivelmente para fins de pesquisa ou correspondência):

In [272]:
t1 = unicodedata.normalize('NFD', s1)
''.join(c for c in t1 if not unicodedata.combining(c))

'Spicy Jalapeno'

Este último exemplo mostra outro aspecto importante do módulo `unicodedata` — ou seja, funções utilitárias para testar caracteres contra classes de caracteres. A função de `combining()` testa um caractere para ver se é um caractere de combinação. 

Existem outras funções no módulo para encontrar categorias de caracteres, testar dígitos e assim por diante. Unicode é obviamente um grande tópico.

## 2.10. Trabalhando com caracteres Unicode em expressões regulares

**Problema**

Você está usando expressões regulares para processar texto, mas está preocupado com o manuseio de caracteres Unicode.

**Solução**

Por padrão, o módulo `re` já está programado com conhecimento rudimentar de certas classes de caracteres Unicode. Por exemplo, `\d` já corresponde a qualquer caractere de dígito unicode:

In [273]:
import re
num = re.compile('\d+')

# Digitos ASCII
num.match('123')

<re.Match object; span=(0, 3), match='123'>

In [274]:
# Digitos Arábicos
num.match('\u0661\u0662\u0663')

<re.Match object; span=(0, 3), match='١٢٣'>

Se você precisar incluir caracteres Unicode específicos em padrões, poderá usar a sequência de escape usual para caracteres Unicode (por exemplo, *\uFFFF* ou *\UFFFFFFF*). Por exemplo, aqui está um regex que corresponde a todos os caracteres em algumas páginas de código árabe diferentes:

In [275]:
arabic = re.compile('[\u0600-\u06ff\u0750-\u077f\u08a0-\u08ff]+')

Ao realizar operações de correspondência e pesquisa, é uma boa ideia normalizar e possivelmente sanear todo o texto para um formulário padrão primeiro (consulte a Receita 2.9). No entanto, também é importante estar atento a casos especiais. Por exemplo, considere o comportamento da correspondência sem distinção entre maiúsculas e minúsculas combinada com a dobragem de maiúsculas:

In [276]:
pat = re.compile('stra\u00dfe', re.IGNORECASE)
s = 'straße'

pat.match(s)

<re.Match object; span=(0, 6), match='straße'>

In [277]:
pat.match(s.upper()) # Não combina

In [278]:
s.upper()

'STRASSE'

**Discussão**

Misturar Unicode e expressões regulares geralmente é uma boa maneira de fazer sua cabeça explodir. Se você for fazer isso com seriedade, considere instalar a biblioteca regex de terceiros, que fornece suporte completo para dobra de maiúsculas e minúsculas Unicode, bem como uma variedade de outros recursos interessantes, incluindo correspondência aproximada.

## 2.11. Tirando caracteres indesejados de strings

**Problema**

Você deseja remover caracteres indesejados, como espaços em branco, do início, fim ou meio de uma sequência de texto.

**Solução**

O método `strip()` pode ser usado para remover caracteres do início ou do final de uma string. `lstrip()` e `rstrip()` executam a remoção do lado esquerdo ou direito, respectivamente. Por padrão, esses métodos removem os espaços em branco, mas outros caracteres podem ser fornecidos. Por exemplo:


In [279]:
# Retirando espaços em branco
s = '    olá povo  \n'

s.strip()

'olá povo'

In [280]:
s.lstrip()

'olá povo  \n'

In [281]:
s.rstrip()

'    olá povo'

In [282]:
# Retirando um caracter específico
t = '----Olá-----'

t.lstrip('-')

'Olá-----'

In [283]:
t.strip('-')

'Olá'

In [284]:
# Cortando o caracter
t = '-----hello====='
t.lstrip('-')

'hello====='

In [285]:
t.strip('-=')

'hello'

**Discussão**


Os vários métodos `strip()`' são comumente usados ao ler e limpar dados para processamento posterior. Por exemplo, você pode usá-los para se livrar de espaços em branco, remover citações e outras tarefas.

Esteja ciente de que a remoção não se aplica a nenhum texto no meio de uma string. Por exemplo:

In [286]:
s = '     olá    pessoal    \n'
s = s.strip()
s

'olá    pessoal'

Se você precisasse fazer algo no espaço interno, precisaria usar outra técnica, como usar o método `replace()` ou uma substituição de expressão regular. Por exemplo:

In [287]:
s.replace(' ','')

'olápessoal'

In [288]:
import re
re.sub('\s+', ' ', s)

'olá pessoal'

Geralmente, você deseja combinar operações de remoção de strings com algum outro tipo de processamento iterativo, como ler linhas de dados de um arquivo. Se sim, esta é uma área onde uma expressão geradora pode ser útil. Por exemplo:
<code>
with open(filename) as f:
    lines = (line.strip() for line in f)
    for line in lines:
        ...
</code>

Aqui, a expressão `lines = (line.strip() for line in f)` atua como um tipo de transformação de dados. É eficiente porque não lê os dados em nenhum tipo de lista temporária primeiro. Ele apenas cria um iterador onde todas as linhas produzidas têm a operação de remoção aplicada a elas.

Para uma remoção ainda mais avançada, você pode recorrer ao método `translate()`.

## 2.12. Sanitização e limpeza de texto

**Problema**

Algum script kiddie entediado inseriu o texto **“pýtĥöñ”** em um formulário em sua página da web e você gostaria de limpá-lo de alguma forma.

**Solução**

O problema de sanitização e limpeza de texto se aplica a uma ampla variedade de problemas envolvendo análise de texto e manipulação de dados. Em um nível muito simples, você pode usar funções básicas de string (por exemplo, `str.upper()` e `str.lower()`) para converter texto em um caso padrão. Substituições simples usando `str.replace()` ou `re.sub()` podem se concentrar na remoção ou alteração de sequências de caracteres muito específicas. Você também pode normalizar o texto usando unicode `data.normalize()`, conforme mostrado na Receita 2.9.

No entanto, você pode querer levar o processo de saneamento um passo adiante. Talvez, por exemplo, você queira eliminar intervalos inteiros de caracteres ou retirar marcas diacríticas. Para fazer isso, você pode recorrer ao método `str.translate()` muitas vezes esquecido. Para ilustrar, suponha que você tenha uma string confusa como a seguinte:

In [289]:
s = 'pýtĥöñ\fis\tawesome\r\n'
s

'pýtĥöñ\x0cis\tawesome\r\n'

O primeiro passo é limpar o espaço em branco. Para fazer isso, faça uma pequena tabela de tradução e use `translate()`:

In [290]:
remap = {
    ord('\t') : ' ',
    ord('\f') : ' ',
    ord('\r') : None # Deletado
}

a = s.translate(remap)
a

'pýtĥöñ is awesome\n'

Como você pode ver aqui, caracteres de espaço em branco como **\t** e **\f** foram remapeados para um único espaço. O retorno de **\r** foi excluído completamente. Você pode levar essa ideia de remapeamento um passo adiante e construir tabelas muito maiores. Por exemplo, vamos remover todos os caracteres combinados:

In [291]:
import unicodedata
import sys

cmb_chrs = dict.fromkeys(c for c in range(sys.maxunicode)
                         if unicodedata.combining(chr(c)))

b = unicodedata.normalize('NFD', a)
b

'pýtĥöñ is awesome\n'

In [292]:
b.translate(cmb_chrs)

'python is awesome\n'

Neste último exemplo, um dicionário mapeando cada caractere de combinação Unicode para **None** é criado usando `dict.fromkeys()`.

A entrada original é então normalizada em uma forma decomposta usando `unicodedata.normalize()`. A partir daí, a função de tradução é usada para excluir todos os acentos. Técnicas semelhantes podem ser usadas para remover outros tipos de caracteres (por exemplo, caracteres de controle, etc.).

Como outro exemplo, aqui está uma tabela de tradução que mapeia todos os caracteres de dígitos decimais Unicode para seu equivalente em ASCII:

In [293]:
digitmap = { c: ord('0') + unicodedata.digit(chr(c)) for c in range(sys.maxunicode)
            if unicodedata.category(chr(c)) == 'Nd' }
len(digitmap)

630

In [294]:
# Dígitos arábicos

x = '\u0661\u0662\u0663'
x.translate(digitmap)

'123'

Ainda outra técnica para limpar texto envolve funções de decodificação e codificação de I/O. A idéia aqui é primeiro fazer uma limpeza preliminar do texto e, em seguida, executá-lo através de uma combinação de operações `encode()` ou `decode()` para remover ou alterá-lo. Por exemplo:

In [295]:
a

'pýtĥöñ is awesome\n'

In [296]:
b = unicodedata.normalize('NFD', a)
b.encode('ascii', 'ignore').decode('ascii')

'python is awesome\n'

Aqui o processo de normalização decompôs o texto original em caracteres juntamente com caracteres de combinação separados. A codificação/decodificação ASCII subsequente simplesmente descartou todos esses caracteres de uma só vez. Naturalmente, isso só funcionaria se obter uma representação ASCII fosse o objetivo final.

**Discussão**

Um grande problema com a limpeza de texto pode ser o desempenho do tempo de execução. *Como regra geral, quanto mais simples, mais rápido será executado*. Para substituições simples, o método `str.replace()` geralmente é a abordagem mais rápida, mesmo que você precise chamá-lo várias vezes. Por exemplo, para limpar o espaço em branco, você pode usar um código como este:

In [297]:
def clean_spaces(s):
    s = s.replace('\r', '')
    s = s.replace('\t', ' ')
    s = s.replace('\f', ' ')
    return s

Se você tentar, verá que é um pouco mais rápido do que usar `translate()` ou uma abordagem usando uma expressão regular.

Por outro lado, o método `translate()` é muito rápido se você precisar executar qualquer tipo de remapeamento ou exclusão não trivial de caractere para caractere.

No quadro geral, o desempenho é algo que você terá que estudar mais em sua aplicação específica. Infelizmente, é impossível sugerir uma técnica específica que funcione melhor para todos os casos, então tente diferentes abordagens e meça-a.

Embora o foco desta receita tenha sido o texto, técnicas semelhantes podem ser aplicadas a bytes, incluindo substituições simples, tradução e expressões regulares.

## 2.13. Alinhando Strings de Texto

**Problema**

Você precisa formatar o texto com algum tipo de alinhamento aplicado.

**Solução**

Para alinhamento básico de strings, os métodos `ljust()`, `rjust()` e `center()` de strings podem ser usados. Por exemplo:

In [298]:
text = 'Olá Pessoal'
text.ljust(20)

'Olá Pessoal         '

In [299]:
text.rjust(20)

'         Olá Pessoal'

In [300]:
text.center(20)

'    Olá Pessoal     '

Todos esses métodos também aceitam um caractere de preenchimento opcional. Por exemplo:

In [301]:
text.rjust(20, '=')

'=========Olá Pessoal'

In [302]:
text.center(20, '=')

'====Olá Pessoal====='

A função `format()` também pode ser usada para alinhar facilmente as coisas. Tudo o que você precisa fazer é usar os caracteres <, > ou ^ junto com a largura desejada. Por exemplo:

In [303]:
format(text, '>20')

'         Olá Pessoal'

In [304]:
format(text, '<20')

'Olá Pessoal         '

In [305]:
format(text, '^20')

'    Olá Pessoal     '

Se você quiser incluir um caractere de preenchimento diferente de um espaço, especifique-o antes do caractere de alinhamento:

In [306]:
format(text, '=>20s')

'=========Olá Pessoal'

In [307]:
format(text, '*^20s')

'****Olá Pessoal*****'

Esses códigos de formato também podem ser usados no método `format()` ao formatar vários valores. Por exemplo:

In [308]:
'{:>10s} {:>10s}'.format('Olá', 'Pessoal')

'       Olá    Pessoal'

Um benefício de `format()` é que ele não é específico para strings. Funciona com qualquer valor, tornando-o mais geral. Por exemplo, você pode usá-lo com números:

In [309]:
x = 1.2345
format(x, '>10')

'    1.2345'

In [310]:
format(x, '^10.2f')

'   1.23   '

**Discussão**

No código mais antigo, você também verá o operador `%` usado para formatar o texto. Por exemplo:

In [311]:
'%20s' % text

'         Olá Pessoal'

In [312]:
'%-20s' % text

'Olá Pessoal         '

No entanto, em um novo código, você provavelmente deve preferir o uso da função ou método `format()`. `format()` é muito mais poderoso do que o fornecido com o operador `%`.

Além disso, `format()` tem um propósito mais geral do que usar o método `jlust()`, `rjust()` ou `center()` de strings, pois funciona com qualquer tipo de objeto.

Para obter uma lista completa de recursos disponíveis com a função `format()`, consulte a documentação online do Python.

## 2.14. Combinando e Concatenando Strings

**Problema**

Você deseja combinar várias pequenas strings em uma string maior.

**Solução**

Se as strings que você deseja combinar forem encontradas em uma sequência ou iterável, a maneira mais rápida de combiná-las é usar o método `join()`. Por exemplo:

In [313]:
parts = ['Is', 'Chicago', 'Not', 'Chicago?']
' '.join(parts)

'Is Chicago Not Chicago?'

In [314]:
','.join(parts)

'Is,Chicago,Not,Chicago?'

In [315]:
''.join(parts)

'IsChicagoNotChicago?'

À primeira vista, essa sintaxe pode parecer muito estranha, mas a operação `join()` é especificada como um método em strings. Em parte, isso ocorre porque os objetos que você deseja unir podem vir de qualquer número de sequências de dados diferentes (por exemplo, listas, tuplas, dicts, arquivos, conjuntos ou geradores), e seria redundante ter `join()` implementado como um método em todos esses objetos separadamente. Então, você apenas especifica a string separadora que deseja e usa o método `join()` nela para colar os fragmentos de texto.

Se você está combinando apenas algumas strings, usar `+` geralmente funciona bem o suficiente:

In [316]:
a = 'Is Chicago'
b = 'Not Chicago?'
a + ' ' + b

'Is Chicago Not Chicago?'

O operador `+` também funciona bem como substituto para operações de formatação de strings mais complicadas. Por exemplo:

In [317]:
print('{} {}'.format(a,b))

Is Chicago Not Chicago?


In [318]:
print(a + ' ' + b )

Is Chicago Not Chicago?


Se você estiver tentando combinar literais de string no código-fonte, basta colocá-los adjacentes um ao outro sem o operador `+`. Por exemplo:

In [319]:
a = 'Olá' 'Pessoal'
a

'OláPessoal'

**Discussão**

Unir strings pode não parecer avançado o suficiente para garantir uma receita inteira, mas geralmente é uma área em que os programadores fazem escolhas de programação que afetam severamente o desempenho de seu código.

A coisa mais importante a saber é que usar o operador `+` para juntar muitas strings é extremamente ineficiente devido às cópias de memória e coleta de lixo que ocorre. Em particular, você nunca quer escrever código que una strings assim:

In [320]:
parts = ['Is', 'Chicago', 'Not', 'Chicago?']
s = ''
for p in parts:
    s +=p

Isso é um pouco mais lento do que usar o método `join()`, principalmente porque cada operação `+=` cria um novo objeto string. É melhor apenas coletar todas as partes primeiro e depois juntá-las no final.

Um truque relacionado (e bem legal) é a conversão de dados em strings e concatenação ao mesmo tempo usando uma expressão geradora, conforme descrito na Receita 1.19. Por exemplo:

In [321]:
data = ['ACME', 50, 91.1]
','.join(str(d) for d in data)

'ACME,50,91.1'

Fique atento também a concatenações de strings desnecessárias. Às vezes, os programadores se empolgam com a concatenação quando não é tecnicamente necessário. Por exemplo, ao imprimir:

In [322]:
a = 'João'
b = 'Carlos'
c = 'Pedro'
print(a + ':' + b + ':' + c) # Feio 
print('')
print(':'.join([a, b, c])) # Ainda feio
print('')
print(a, b, c, sep=':') # Melhor um pouco!!!

João:Carlos:Pedro

João:Carlos:Pedro

João:Carlos:Pedro


Misturar operações de I/O e concatenação de strings é algo que pode exigir estudo em sua aplicação. Por exemplo, considere os dois fragmentos de código a seguir:

<code>
\# Versão 1 (concatenação de strings)
f.write(chunk1 + chunk2)
    
\# Versão 2 (operações de E/S separadas)
f.write(chunk1)
f.write(chunk2)
</code>

Se as duas strings forem pequenas, a primeira versão poderá oferecer um desempenho muito melhor devido ao custo inerente de realizar uma chamada de sistema de I/O. Por outro lado, se as duas strings forem grandes, a segunda versão pode ser mais eficiente, pois evita fazer um grande resultado temporário e copiar grandes blocos de memória. Novamente, deve-se enfatizar que isso é algo que você teria que estudar em relação aos seus próprios dados para determinar qual tem o melhor desempenho.

Por último, mas não menos importante, se você estiver escrevendo código que está construindo saída de muitas pequenas strings, considere escrever esse código como uma função geradora, usando `yield` para emitir fragmentos. Por exemplo:

In [323]:
def sample():
    yield 'Is'
    yield 'Chicago'
    yield 'Not'
    yield 'Chicago?'

O interessante dessa abordagem é que ela não faz nenhuma suposição sobre como os fragmentos devem ser reunidos. Por exemplo, você pode simplesmente juntar os fragmentos usando `join()`:

In [324]:
' '.join(sample())

'Is Chicago Not Chicago?'

Ou você pode redirecionar os fragmentos para I/O:
<code>
for part in sample():
    f.write(part)
</code>

Ou você pode criar algum tipo de esquema híbrido que seja inteligente para combinar operações de I/O:

<code>
def combine(source, maxsize):
    parts = []
    size = 0
    for part in source:
        parts.append(part)
        size += len(part)
        if size > maxsize:
            yield ''.join(parts)
            parts = []
            size = 0
    yield ''.join(parts)

for part in combine(sample(), 32768):
    f.write(part)   
</code>

O ponto chave é que a função geradora original não precisa conhecer os detalhes precisos. Ele apenas cede as partes.

## 2.15. Interpolando Variáveis em Strings

**Problema**

Você deseja criar uma string na qual os nomes das variáveis incorporadas sejam substituídos por uma representação em string do valor de uma variável.

**Solução**

Python não tem suporte direto para simplesmente substituir valores de variáveis em strings. No entanto, esse recurso pode ser aproximado usando o método `format()` de strings. Por exemplo:

In [325]:
s = '{name} has {n} mensages.'
s.format(name='João', n=40)

'João has 40 mensages.'

Alternativamente, se os valores a serem substituídos forem realmente encontrados em variáveis, você pode usar a combinação de `format_map()` e `vars()`, como a seguir:

In [326]:
name = 'João'
n = 40

s.format_map(vars())

'João has 40 mensages.'

Um recurso sutil de `vars()` é que ele também funciona com instâncias. Por exemplo:

In [327]:
class Info:
    def __init__(self, name, n):
        self.name = name
        self.n = n

a = Info('João',40)
s.format_map(vars(a))

'João has 40 mensages.'

Uma desvantagem de `format()` e `format_map()` é que eles não lidam bem com valores ausentes. Por exemplo:

<code>
>>> s.format(name='Guido')
Traceback (most recent call last):
    File "<stdin>", line 1, in <module>
KeyError: 'n'
>>>
</code>

Uma maneira de evitar isso é definir uma classe de dicionário alternativa com um método `__missing__()`, como no seguinte:

In [328]:
class safesub(dict):
    def __missing__(self, key):
        return '{' + key + '}'


Agora use esta classe para envolver as entradas para `format_map()`:

In [329]:
del n # Tenha certeza que <n> está indefinido

In [330]:
s.format_map(safesub(vars()))

'João has {n} mensages.'

Se você estiver executando essas etapas com frequência em seu código, poderá ocultar o processo de substituição de variável por trás de uma pequena função de utilitário que emprega o chamado “hack de quadro”. Por exemplo:

In [331]:
import sys

def sub(text):
    return text.format_map(safesub(sys._getframe(1).f_locals))

Agora você pode digitar coisas assim:

In [332]:
name = 'João'
n = 37

print(sub('Hello {name}'))

Hello João


In [333]:
print(sub('Sua cor favorita é {color}'))

Sua cor favorita é {color}


**Discussão**

A falta de interpolação de variáveis verdadeiras no *Python* levou a uma variedade de soluções ao longo dos anos. Você também pode ver o uso de strings de modelo:

In [334]:
name = 'João'
n = 40
import string
s = string.Template('$name has $n messages.')
s.substitute(vars())

'João has 40 messages.'

No entanto, os métodos `format()` e `format_map()` são mais modernos que qualquer uma dessas alternativas e devem ser preferidos. Um benefício de usar `format()` é que você também obtém todos os recursos relacionados à formatação de string (alinhamento, preenchimento, formatação numérica etc.), o que simplesmente não é possível com alternativas como objetos de string de modelo.

Partes desta receita também ilustram alguns recursos avançados interessantes. O método pouco conhecido `__missing__()` de classes de mapeamento/dict é um método que você pode definir para lidar com valores ausentes. Na classe `safesub`, esse método foi definido para retornar valores ausentes como um espaço reservado. Em vez de obter uma exceção `KeyError`, você veria os valores ausentes aparecendo na string resultante (potencialmente útil para depuração).

A função `sub()` usa `sys._getframe(1)` para retornar o quadro de pilha do chamador. A partir disso, o atributo `f_locals` é acessado para obter as variáveis locais. Isso sem dizer que, com quadros de pilha provavelmente deve ser evitado na maioria dos códigos. No entanto, para funções utilitárias, como um recurso de substituição de string, pode ser útil. Como um aparte, provavelmente vale a pena notar que `f_locals` é um dicionário que é uma cópia das variáveis locais na função de chamada. Embora você possa modificar o conteúdo de `f_locals`, as modificações não têm nenhum efeito duradouro. Assim, mesmo que acessar um frame de pilha diferente possa parecer ruim, não é possível sobrescrever variáveis acidentalmente ou alterar o ambiente local do chamador.

## 2.16. Reformatando o texto para um número fixo de colunas

**Problema**

Você tem strings longas que deseja reformatar para que elas preencham um número de colunas especificado pelo usuário.

**Solução**

Use o módulo `textwrap` para reformatar o texto para saída. Por exemplo, suponha que você tenha a seguinte string longa:

In [335]:
s = "Look into my eyes, look into my eyes, the eyes, the eyes, \
the eyes, not around the eyes, don't look around the eyes, \
look into my eyes, you're under."

Veja como você pode usar o módulo `textwrap` para reformatá-lo de várias maneiras:

In [336]:
import textwrap
print(textwrap.fill(s, 70))

Look into my eyes, look into my eyes, the eyes, the eyes, the eyes,
not around the eyes, don't look around the eyes, look into my eyes,
you're under.


In [337]:
print(textwrap.fill(s, 40))

Look into my eyes, look into my eyes,
the eyes, the eyes, the eyes, not around
the eyes, don't look around the eyes,
look into my eyes, you're under.


In [338]:
print(textwrap.fill(s, 40, initial_indent='  '))

  Look into my eyes, look into my eyes,
the eyes, the eyes, the eyes, not around
the eyes, don't look around the eyes,
look into my eyes, you're under.


In [339]:
print(textwrap.fill(s, 40, subsequent_indent=' '))

Look into my eyes, look into my eyes,
 the eyes, the eyes, the eyes, not
 around the eyes, don't look around the
 eyes, look into my eyes, you're under.


**Discussão**

O módulo `textwrap` é uma maneira direta de limpar o texto para impressão, especialmente, se você quiser que a saída caiba bem no terminal. Sobre o tamanho do terminal, você pode obtê-lo usando `os.get_terminal_size()`. Por exemplo:

In [340]:
import os
os.get_terminal_size().columns

120

In [341]:
n = os.get_terminal_size().columns
print(textwrap.fill(s, n))

Look into my eyes, look into my eyes, the eyes, the eyes, the eyes, not around the eyes, don't look around the eyes,
look into my eyes, you're under.


O método `fill()` tem algumas opções adicionais que controlam como ele lida com tabulações, terminações de frases e assim por diante. Consulte a documentação da classe `textwrap.TextWrapper` para obter mais detalhes.

## 2.17. Manipulando Entidades HTML e XML em Texto

**Problema**

Você deseja substituir entidades HTML ou XML como **&entity**; ou **&code**; com o texto correspondente. Como alternativa, você precisa produzir texto, mas escapar de certos caracteres (por exemplo, <, > ou &).

**Solução**

Se você estiver produzindo texto, substituir caracteres especiais como < ou > é relativamente fácil se você usar a função `html.escape()`. Por exemplo:

In [342]:
s = 'Elements are written as "<tag>text</tag>".'
print(s)

Elements are written as "<tag>text</tag>".


In [343]:
import html
print(html.escape(s))

Elements are written as &quot;&lt;tag&gt;text&lt;/tag&gt;&quot;.


In [344]:
# Disabilitar o escape das Quotes
print(html.escape(s,quote=False))

Elements are written as "&lt;tag&gt;text&lt;/tag&gt;".


Se você está tentando emitir texto como ASCII e deseja incorporar entidades de código de caracteres para caracteres não ASCII, você pode usar o argumento `errors='xmlcharrefreplace'` para várias funções relacionadas a I/O para fazer isso. Por exemplo:

In [345]:
s = 'Spicy Jalapeño'
s.encode('ascii', errors='xmlcharrefreplace')

b'Spicy Jalape&#241;o'

Para substituir entidades no texto, é necessária uma abordagem diferente. Se você estiver realmente processando HTML ou XML, tente primeiro usar um analisador HTML ou XML adequado. Normalmente, essas ferramentas cuidarão automaticamente da substituição dos valores para você durante a análise e você não precisa se preocupar com isso.

Se, por algum motivo, você recebeu um texto simples com algumas entidades e deseja que elas sejam substituídas manualmente, geralmente você pode fazer isso usando várias funções/métodos de utilitários associados a analisadores HTML ou XML. Por exemplo:

In [346]:
s = 'Spicy &quot;Jalape&#241;o&quot.'

from html import unescape

unescape(s)

'Spicy "Jalapeño".'

In [347]:
t = 'The prompt is &gt;&gt;&gt;'

from xml.sax.saxutils import unescape
unescape(t)

'The prompt is >>>'

**Discussão**

O escape adequado de caracteres especiais é um detalhe facilmente esquecido na geração de HTML ou XML. Isso é especialmente verdadeiro se você estiver gerando essa saída usando `print()` ou outros recursos básicos de formatação de string. Usar uma função utilitária como `html.escape()` é uma solução fácil.

Se você precisar processar texto em outra direção, várias funções de utilitário, como `xml.sax.saxutils.unescape()`, podem ajudar. No entanto, você realmente precisa investigar o uso de um analisador adequado. Por exemplo, se estiver processando HTML ou XML, usar um módulo de análise como `html.parser` ou `xml.etree.ElementTree` já deve cuidar dos detalhes relacionados à substituição de entidades no texto de entrada para você.

## 2.18. Texto de tokenização

**Problema**

Você tem uma string que deseja analisar da esquerda para a direita em um fluxo de tokens.

**Solução**

Suponha que você tenha uma string de texto como esta:

In [348]:
text = 'foo = 23 + 42 * 10'

Para tokenizar a string, você precisa fazer mais do que apenas combinar padrões. Você precisa ter alguma maneira de identificar o tipo de padrão também. Por exemplo, você pode querer transformar a string em uma sequência de pares como esta:

In [349]:
tokens = [
    ('NAME', 'foo'),
    ('EQ','='),
    ('NUM', '23'),
    ('PLUS','+'),
    ('NUM', '42'),
    ('TIMES', '*'),
    ('NUM', 10)
]

Para fazer esse tipo de divisão, o primeiro passo é definir todos os tokens possíveis, incluindo espaços em branco, por padrões de expressão regular usando grupos de captura nomeados como este:

In [350]:
import re

NAME = r'(?P<NAME>[a-zA-Z_][a-zA-Z_0-9]*)'
NUM = r'(?P<NUM>\d+)'
PLUS = r'(?P<PLUS>\+)'
TIMES = r'(?P<TIMES>\*)'
EQ = r'(?P<EQ>=)'
WS = r'(?P<WS>\s+)'

In [351]:
master_pat = re.compile('|'.join([NAME, NUM, PLUS, TIMES, EQ, WS]))

Nesses padrões, a convenção **\?P\<TOKENNAME>** é usada para atribuir um nome ao padrão. Isso será usado mais tarde.
Em seguida, para tokenizar, use o método pouco conhecido `scanner()` de objetos padrão. Esse método cria um objeto scanner no qual chamadas repetidas para `match()` percorrem o texto fornecido, uma correspondência por vez. Aqui está um exemplo interativo de como um objeto scanner funciona:

In [352]:
scanner = master_pat.scanner('foo = 42')
scanner.match()

<re.Match object; span=(0, 3), match='foo'>

In [353]:
_.lastgroup, _.group()

('NAME', 'foo')

In [354]:
scanner.match()

<re.Match object; span=(3, 4), match=' '>

In [355]:
_.lastgroup, _.group()

('WS', ' ')

In [356]:
scanner.match()

<re.Match object; span=(4, 5), match='='>

In [357]:
_.lastgroup, _.group()

('EQ', '=')

In [358]:
scanner.match()

<re.Match object; span=(5, 6), match=' '>

In [359]:
_.lastgroup, _.group()

('WS', ' ')

In [360]:
scanner.match()

<re.Match object; span=(6, 8), match='42'>

In [361]:
_.lastgroup, _.group()

('NUM', '42')

In [362]:
scanner.match()

Para pegar essa técnica e colocá-la em código, ela pode ser limpa e facilmente empacotada em um gerador como este:

In [363]:
from collections import namedtuple

Token = namedtuple('Token', ['type','value'])

def generate_tokens(pat, text):
    scanner = pat.scanner(text)
    for m in iter(scanner.match, None):
        yield Token(m.lastgroup, m.group())
        
# Examplo de uso 
for tok in generate_tokens(master_pat, 'foo = 42'):
    print(tok)

Token(type='NAME', value='foo')
Token(type='WS', value=' ')
Token(type='EQ', value='=')
Token(type='WS', value=' ')
Token(type='NUM', value='42')


Se você deseja filtrar o fluxo de tokens de alguma forma, você pode definir mais funções geradoras ou usar uma expressão geradora. Por exemplo, aqui está como você pode filtrar todos os tokens de espaço em branco.

In [364]:
tokens = (tok for tok in generate_tokens(master_pat, text) if tok.type != 'WS')
for tok in tokens:
    print(tok)

Token(type='NAME', value='foo')
Token(type='EQ', value='=')
Token(type='NUM', value='23')
Token(type='PLUS', value='+')
Token(type='NUM', value='42')
Token(type='TIMES', value='*')
Token(type='NUM', value='10')


**Discussão**

A tokenização geralmente é o primeiro passo para tipos mais avançados de análise e manipulação de texto. Para usar a técnica de digitalização mostrada, há alguns detalhes importantes a serem lembrados. Primeiro, você deve certificar-se de identificar todas as sequências de texto possíveis que possam aparecer na entrada com um padrão `re` correspondente. Se algum texto não correspondente for encontrado, a varredura simplesmente será interrompida. É por isso que foi necessário especificar o token de espaço em branco (WS) no exemplo.

A ordem dos tokens na expressão regular mestre também importa. Ao fazer a correspondência, tenta corresponder os padrões na ordem especificada. Assim, se um padrão for uma subcadeia de um padrão mais longo, você precisa garantir que o padrão mais longo seja o primeiro. Por exemplo:

In [365]:
LT = r'(?P<LT><)'
LE = r'(?P<LE><=)'
EQ = r'(?P<EQ>=)'

In [366]:
master_pat = re.compile('|'.join([LE, LT, EQ])) # Correct
# master_pat = re.compile('|'.join([LT, LE, EQ])) # Incorrect

In [367]:
master_pat

re.compile(r'(?P<LE><=)|(?P<LT><)|(?P<EQ>=)', re.UNICODE)

O segundo padrão está errado porque corresponderia ao texto **<=** como o token LT seguido pelo token EQ, não o token único LE, como provavelmente foi desejado.

Por último, mas não menos importante, você precisa ficar atento aos padrões que formam substrings. Por exemplo, suponha que você tenha dois padrões como este:

In [368]:
PRINT = r'(P<PRINT>print)'
NAME = r'(P<NAME>[a-zA-Z_][a-zA-Z_0-9]*)'

master_pat = re.compile('|'.join([PRINT, NAME]))

for tok in generate_tokens(master_pat, 'printer'):
    print(tok)

Para tipos mais avançados de tokenização, você pode verificar pacotes como PyParsing ou PLY. Um exemplo envolvendo PLY aparece na próxima receita.

## 2.19. Escrevendo um analisador de descida recursiva simples

**Problema**

Você precisa analisar o texto de acordo com um conjunto de regras gramaticais e executar ações ou construir uma árvore de sintaxe abstrata representando a entrada. A gramática é pequena, então você prefere apenas escrever o analisador em vez de usar algum tipo de estrutura.

**Solução**

Neste problema, estamos focados no problema de analisar o texto de acordo com uma gramática específica. Para fazer isso, você provavelmente deve começar por ter uma especificação formal da gramática na forma de uma BNF ou EBNF. Por exemplo, uma gramática para expressões aritméticas simples pode ser assim:

<code>
expr ::= expr + term
     | expr - term
     | term
term ::= term * factor
     | term / factor
     | factor
factor ::= ( expr )
       | NUM
</code>

Ou, alternativamente, na forma EBNF:

<code>
expr ::= term { (+|-) term }*

term ::= factor { (*|/) factor }*

factor ::= ( expr )
        | NUM
</code>

Em uma EBNF, partes de uma regra entre { ... }* são opcionais. O * significa zero ou mais repetições (o mesmo significado que em uma expressão regular).

Agora, se você não estiver familiarizado com a mecânica de trabalhar com uma BNF, pense nisso como uma especificação de substituição ou regras de substituição onde os símbolos do lado esquerdo podem ser substituídos pelos símbolos do lado direito (ou vice-versa). Geralmente, o que acontece durante a análise é que você tenta combinar o texto de entrada com a gramática fazendo várias substituições e expansões usando o BNF. Para ilustrar, suponha que você esteja analisando uma expressão como **3 + 4 * 5**. Essa expressão precisaria primeiro ser dividida em um fluxo de token, usando as técnicas descritas na Receita 2.18. O resultado pode ser uma sequência de tokens como esta:
<code>
    NUM + NUM * NUM
</code>

A partir daí, a análise envolve tentar combinar a gramática com os tokens de entrada fazendo substituições:
<code>
exp
expr ::= termo { (+|-) termo }*
expr ::= fator { (*|/) fator }* { (+|-) termo }*
expr ::= NUM { (*|/) fator }* { (+|-) termo }*
expr ::= NUM { (+|-) termo }*
expr ::= NUM + termo { (+|-) termo }*
expr ::= NUM + fator { (*|/) fator }* { (+|-) termo }*
expr ::= NUM + NUM { (*|/) fator}* { (+|-) termo }*
expr ::= NUM + NUM * fator { (*|/) fator }* { (+|-) termo }*
expr ::= NUM + NUM * NUM { (*|/) fator }* { (+|-) termo }*
expr ::= NUM + NUM * NUM { (+|-) termo }*
expr ::= NUM + NUM * NUM
</code>

Seguir todas as etapas de substituição exige um pouco de café, mas eles são orientados pela observação da entrada e pela tentativa de combiná-la com as regras gramaticais. O primeiro token de entrada é um NUM, portanto, as substituições primeiro se concentram na correspondência dessa parte. Uma vez correspondido, a atenção se move para o próximo token de + e assim por diante. Certas partes do lado direito (por exemplo, **{ (\*/) factor }*)** desaparecem quando é determinado que não podem corresponder ao próximo token. Em uma análise bem-sucedida, todo o lado direito é expandido completamente para corresponder ao fluxo de token de entrada.

Com todos os antecedentes anteriores no lugar, aqui está uma receita simples que mostra como construir um avaliador de expressão descendente recursiva:

In [371]:
import re
import collections

# Token specification
NUM = r'(?P<NUM>\d+)'
PLUS = r'(?P<PLUS>\+)'
MINUS = r'(?P<MINUS>-)'
TIMES = r'(?P<TIMES>\*)'
DIVIDE = r'(?P<DIVIDE>/)'
LPAREN = r'(?P<LPAREN>\()'
RPAREN = r'(?P<RPAREN>\))'
WS = r'(?P<WS>\s+)'

master_pat = re.compile('|'.join([NUM, PLUS, MINUS, TIMES,DIVIDE, LPAREN, RPAREN, WS]))

# Tokenizer
Token = collections.namedtuple('Token', ['type','value'])

def generate_tokens(text):
    scanner = master_pat.scanner(text)
    for m in iter(scanner.match, None):
        tok = Token(m.lastgroup, m.group())
        if tok.type != 'WS':
            yield tok

# Parser
class ExpressionEvaluator:
        '''
        Implementation of a recursive descent parser. Each method
        implements a single grammar rule. Use the ._accept() method
        to test and accept the current lookahead token. Use the ._expect()
        method to exactly match and discard the next token on on the input
        (or raise a SyntaxError if it doesn't match).
        '''
        def parse(self,text):
            self.tokens = generate_tokens(text)
            self.tok = None                    # Last symbol consumed
            self.nexttok = None                # Next symbol tokenized
            self._advance()                    # Load first lookahead token
            return self.expr()
        
        def _advance(self):
            'Advance one token ahead'
            self.tok, self.nexttok = self.nexttok, next(self.tokens, None)
        
        def _accept(self,toktype):
            'Test and consume the next token if it matches toktype'
            if self.nexttok and self.nexttok.type == toktype:
                self._advance()
                return True            
            else:
                return False
    
        def _expect(self,toktype):
            'Consume next token if it matches toktype or raise SyntaxError'
            if not self._accept(toktype):
                raise SyntaxError('Expected ' + toktype)

    # Grammar rules follow
        def expr(self):
            "expression ::= term { ('+'|'-') term }*"
            exprval = self.term()
            while self._accept('PLUS') or self._accept('MINUS'):
                op = self.tok.type
                right = self.term()
                if op == 'PLUS':
                    exprval += right
                elif op == 'MINUS':
                    exprval -= right
            return exprval
    
        def term(self):
            "term ::= factor { ('*'|'/') factor }*"
            termval = self.factor()
            while self._accept('TIMES') or self._accept('DIVIDE'):
                op = self.tok.type
                right = self.factor()
                if op == 'TIMES':
                    termval *= right
                elif op == 'DIVIDE':
                    termval /= right
            return termval

        def factor(self):
            "factor ::= NUM | ( expr )"
            if self._accept('NUM'):
                return int(self.tok.value)
            elif self._accept('LPAREN'):
                exprval = self.expr()
                self._expect('RPAREN')
                return exprval
            else:
                raise SyntaxError('Expected NUMBER or LPAREN')            

Aqui está um exemplo de uso da classe `ExpressionEvaluator` interativamente:

In [373]:
e = ExpressionEvaluator()
e.parse('2')


2

In [374]:
e.parse('2 + 3')

5

In [375]:
e.parse('2 + 3 * 4')

14

In [376]:
e.parse('2 + (3 +4) * 5')

37

Se você quiser fazer algo diferente da avaliação pura, precisará alterar a classe `ExpressionEvaluator` para fazer outra coisa. Por exemplo, aqui está uma implementação alternativa que constrói uma árvore de análise simples:

In [377]:
class ExpressionTreeBuilder(ExpressionEvaluator):
    def expr(self):
        "expression ::= term { ('+'|'-') term }"
        exprval = self.term()
        while self._accept('PLUS') or self._accept('MINUS'):
            op = self.tok.type
            right = self.term()
            if op == 'PLUS':
                exprval = ('+', exprval, right)
            elif op == 'MINUS':
                exprval = ('-', exprval, right)
        return exprval
    
    def term(self):
        "term ::= factor { ('*'|'/') factor }"
        termval = self.factor()
        while self._accept('TIMES') or self._accept('DIVIDE'):
            op = self.tok.type
            right = self.factor()
            if op == 'TIMES':
                termval = ('*', termval, right)
            elif op == 'DIVIDE':
                termval = ('/', termval, right)
        return termval
    
    def factor(self):
        'factor ::= NUM | ( expr )'
        if self._accept('NUM'):
            return int(self.tok.value)
        elif self._accept('LPAREN'):
            exprval = self.expr()
            self._expect('RPAREN')
            return exprval
        else:
            raise SyntaxError('Expected NUMBER or LPAREN')

O exemplo a seguir mostra como funciona:

In [379]:
e = ExpressionTreeBuilder()

In [380]:
e.parse('2 + 3')

('+', 2, 3)

In [381]:
e.parse('2 + 3 * 4')

('+', 2, ('*', 3, 4))

In [382]:
e.parse('2 + (3 + 4) * 5')

('+', 2, ('*', ('+', 3, 4), 5))

In [383]:
e.parse('2 + 3 +4')

('+', ('+', 2, 3), 4)

**Discussão**

*Parsing* é um tópico enorme que geralmente ocupa os alunos nas primeiras três semanas de um curso de compiladores. Se você está procurando conhecimento básico sobre gramáticas, algoritmos de análise e outras informações, um livro de compiladores é o que você deve procurar. 

No entanto, a ideia geral de escrever um analisador descendente recursivo é geralmente simples. Para começar, você pega todas as regras gramaticais e as transforma em uma função ou método. Assim, se sua gramática se parece com isso:

<code>
expr ::= termo { ('+'|'-') term }*
termo ::= fator { ('*'|'/') factor }*
fator ::= '('expr')' 
      |  NUM
</code>

Você começa transformando-o em um conjunto de métodos como este:

<code>
class ExpressionEvaluator:
    ...
    def expr(self):
        ...
    def term (self):
        ... 
    def factor(self):
        ...
</code>

A tarefa de cada método é simples – ele deve andar da esquerda para a direita sobre cada parte da regra gramatical, consumindo tokens no processo. De certa forma, o objetivo do método é consumir a regra ou gerar um erro de sintaxe se ela travar. Para fazer isso, as seguintes técnicas de implementação são aplicadas:

* Se o próximo símbolo na regra for o nome de outra regra gramatical (por exemplo, termo ou fator), basta chamar o método com o mesmo nome. Esta é a parte “descendente” do algoritmo – o controle desce para outra regra gramatical. Às vezes, as regras envolvem chamadas para métodos que já estão em execução (por exemplo, a chamada para expr na regra fator ::= '(' expr ')'). Esta é a parte “recursiva” do algoritmo.
* Se o próximo símbolo na regra tiver que ser um símbolo específico (por exemplo, (), você olha para o próximo token e verifica se há uma correspondência exata. Se não corresponder, é um erro de sintaxe. O método `_expect()` nesta receita é usado para executar essas etapas.
* Se o próximo símbolo na regra puder ser algumas escolhas possíveis (por exemplo, + ou -), você deve verificar o próximo símbolo para cada possibilidade e avançar apenas se houver uma correspondência. Este é o propósito do método `_accept()` nesta receita. É como uma versão mais fraca do método `_expect()` na medida em que avançará se uma correspondência for feita, mas se não for, simplesmente recuará sem gerar um erro (permitindo assim que outras verificações sejam feitas).
* Para regras gramaticais onde há partes repetidas (por exemplo, como na regra expr ::= term { ('+'|'-') term }\*), a repetição é implementada por um loop while. O corpo do loop geralmente coleta ou processa todos os itens repetidos até não são mais encontrados.
* Uma vez que uma regra gramatical inteira foi consumida, cada método retorna algum tipo do resultado de volta para o chamador. É assim que os valores se propagam durante a análise. Por exemplo, no avaliador de expressão, os valores de retorno representarão resultados parciais da expressão que está sendo analisada. Eventualmente, todos eles são combinados no mais alto método de regra gramatical que é executado.

Embora um exemplo simples tenha sido mostrado, analisadores de descida recursiva podem ser usados para implementar analisadores bastante complicados. Por exemplo, o próprio código Python é interpretado por um analisador descendente recursivo. Se você estiver tão inclinado, você pode olhar para a gramática subjacente inspecionando o arquivo *Grammar/Grammar* na fonte Python. Dito isso, há ainda existem inúmeras armadilhas e limitações com a criação de um analisador à mão.


Uma dessas limitações dos analisadores de descendência recursiva é que eles não podem ser escritos para gramática regras envolvendo qualquer tipo de recursão à esquerda. Por exemplo, suponha que você precise traduzir um regra assim:

<code>
itens ::= itens ',' item
       | item
</code>

Para fazer isso, você pode tentar usar o método `items()` assim:

In [384]:
def items(self):
    itemsval = self.items()
    if itemsval and self._accept(','):
        itemsval.append(self.item())
    else:
        itemsval = [ self.item() ]

O único problema é que não funciona. Na verdade, ele explode com um erro de recursão infinito.

Você também pode se deparar com problemas complicados relacionados às próprias regras gramaticais. Por exemplo, você pode ter se perguntado se as expressões poderiam ou não ter sido descritas por esta gramática mais simples:

<code>
expr ::= factor { ('+'|'-'|'*'|'/') factor }*
    
fator ::= '(' expressão ')'
    | NUM
</code>

Essa gramática tecnicamente “funciona”, mas não observa as regras aritméticas padrão relativas à ordem de avaliação. Por exemplo, a expressão “3 + 4 * 5” seria avaliada como “35” em vez do resultado esperado de “23”. O uso de regras separadas “expr” e “term” existe para que a avaliação funcione corretamente.

Para gramáticas realmente complicadas, geralmente é melhor usar ferramentas de análise sintática, como PyParsing ou PLY.

## 2.20. Executando operações de texto em strings de bytes

**Problema**

Você deseja realizar operações de texto comuns (por exemplo, remoção, pesquisa e substituição) em cadeias de bytes.

**Solução**

Cadeias de bytes já suportam a maioria das mesmas operações internas que cadeias de texto. Por exemplo:

In [399]:
data = b'Hello World'
data[0:5]

b'Hello'

In [400]:
data.startswith(b'Hello')

True

In [401]:
data.split()

[b'Hello', b'World']

In [405]:
data.replace(b'Hello', b'Hello Big')

b'Hello Big World'

Essas operações também funcionam com matrizes de bytes. Por exemplo:

In [406]:
data = bytearray(b'Hello World')
data[0:5]

bytearray(b'Hello')

In [408]:
data.startswith(b'Hello')

True

In [410]:
data.split()

[bytearray(b'Hello'), bytearray(b'World')]

In [411]:
data.replace(b'Hello', b'Hello Big')

bytearray(b'Hello Big World')

Você pode aplicar a correspondência de padrões de expressão regular a cadeias de bytes, mas os próprios padrões precisam ser especificados como bytes. Por exemplo:

In [412]:
data = b'FOO:BAR,SPAM'

In [413]:
import re
re.split('[:,]', data)

TypeError: cannot use a string pattern on a bytes-like object

In [415]:
re.split(b'[:,]', data) # Padrão em Bytes

[b'FOO', b'BAR', b'SPAM']

**Discussão**

Na maioria das vezes, quase todas as operações disponíveis em strings de texto funcionarão em strings de bytes. No entanto, existem algumas diferenças notáveis a serem observadas. Primeiro, a indexação de strings de bytes produz inteiros, não caracteres individuais. Por exemplo:

In [416]:
a = 'Hello World' # Text string
a[0]

'H'

In [417]:
a[1]

'e'

In [418]:
b = b'Hello World' # String em Byte
b[0]


72

In [419]:
b[1]

101

Essa diferença na semântica pode afetar programas que tentam processar dados orientados a byte caractere por caractere.

Em segundo lugar, as strings de bytes não fornecem uma boa representação de string e não são impressas de forma limpa, a menos que primeiro sejam decodificadas em uma string de texto. Por exemplo:

In [421]:
s = b'Hello World'
print(s)

b'Hello World'


In [422]:
print(s.decode('ascii'))

Hello World


Realizando a formatação de strings disponíveis para strings de bytes:

In [423]:
b'%10s %10d %10.2f' % (b'ACME', 100, 490.1)

b'      ACME        100     490.10'

In [424]:
# Este o Python não suporta por ser bytes.
b'{} {} {}'.format(b'ACME', 100, 490.1)

AttributeError: 'bytes' object has no attribute 'format'

Se você deseja fazer qualquer tipo de formatação aplicada a strings de bytes, isso deve ser feito usando strings de texto e codificação normais. Por exemplo:

In [425]:
'{:10s} {:10d} {:10.2f}'.format('ACME', 100, 490.1).encode('ascii')

b'ACME              100     490.10'

Finalmente, você precisa estar ciente de que usar uma string de bytes pode alterar a semântica de certas operações—especialmente aquelas relacionadas ao sistema de arquivos. Por exemplo, se você fornecer um nome de arquivo codificado como bytes em vez de uma string de texto, geralmente desativa a codificação/decodificação de nome de arquivo. Por exemplo:

In [428]:
# Write a UTF-8 filename
with open('jalape\xf1o.txt', 'w') as f:
    f.write('spicy')

# Get a directory listing
import os
os.listdir('.') # Text string (names are decoded)

['.git',
 '.idea',
 '.ipynb_checkpoints',
 'Capítulo 1 - Estruturas de Dados e Algoritmos.ipynb',
 'Capítulo 2 - Strings e Texto.ipynb',
 'jalapeño.txt',
 'LICENSE',
 'README.md',
 'somefile.txt']

In [429]:
os.listdir(b'.')

[b'.git',
 b'.idea',
 b'.ipynb_checkpoints',
 b'Cap\xc3\xadtulo 1 - Estruturas de Dados e Algoritmos.ipynb',
 b'Cap\xc3\xadtulo 2 - Strings e Texto.ipynb',
 b'jalape\xc3\xb1o.txt',
 b'LICENSE',
 b'README.md',
 b'somefile.txt']

Observe na última parte deste exemplo como fornecer uma string de bytes como o nome do diretório fez com que os nomes de arquivos resultantes fossem retornados como bytes não codificados. O nome do arquivo mostrado na listagem de diretórios contém codificação UTF-8 bruta. Veja a Receita 5.15 para alguns problemas relacionados a nomes de arquivos. 

Como comentário final, alguns programadores podem estar inclinados a usar strings de bytes como alternativa às strings de texto devido a uma possível melhoria de desempenho. Embora seja verdade que a manipulação de bytes tende a ser um pouco mais eficiente que o texto (devido à sobrecarga inerente relacionada ao Unicode), isso geralmente leva a um código muito confuso e não idiomático. Muitas vezes você descobrirá que strings de bytes não funcionam bem com muitas outras partes do Python e que você acaba tendo que realizar todos os tipos de operações manuais de codificação/decodificação para fazer as coisas funcionarem corretamente. Francamente, se você estiver trabalhando com texto, use strings de texto normais em seu programa, não strings de bytes.